江苏省建筑市场监管与诚信信息一体化平台
http://58.213.147.230:7001/Jsjzyxyglpt/faces/public/projectList.jsp

In [1]:
from IPython.display import Image
import requests, json, html5lib, time
from urllib.parse import urlencode
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# 测试用代理设置
proxies={
    'http':'211.139.119.100:18080',
}
proxies={}

In [3]:
# Http Headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Accept': 'text/html, application/xhtml+xml, image/jxr, */*',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Referer': 'http://58.213.147.230:7001/Jsjzyxyglpt/faces/public/projectList.jsp',
    'DNT' : '1',
    'Pragma':'no-cache',    
}

In [4]:
# 保存session
reqs = requests.session()

In [5]:
#首次打开，输入验证码并获取结果
url_get_first = 'http://58.213.147.230:7001/Jsjzyxyglpt/faces/public/projectList.jsp?projectNodeType=builderLicence'

In [6]:
resp_first = reqs.get(url_get_first, headers=headers, proxies=proxies)

In [7]:
# 获取 "com.sun.faces.VIEW"
soup_first = BeautifulSoup(resp_first.content.decode(encoding='gbk'),"lxml")
post_com_sun_faces_VIEW = soup_first.find_all(attrs={'name':'com.sun.faces.VIEW'})[0].attrs['value']

In [8]:
# 处理验证码
url_checkCode = "http://58.213.147.230:7001/Jsjzyxyglpt/faces/public/checkCode.jsp"
resp_checkCode = reqs.get(url_checkCode,headers=headers, proxies=proxies)

display(Image(resp_checkCode.content))

str_checkCode = input("输入验证码：")

post_data_checkCode = {
    'action':'isValidCheckCode',
    'checkCode':str_checkCode
}

url_checkCode_action = "http://58.213.147.230:7001/Jsjzyxyglpt/faces/public/action.jsp"
reqs.post(url_checkCode_action,post_data_checkCode, headers=headers, proxies=proxies).content

输入验证码：nafk


b'{"validCheckCode":true}'

In [9]:
# 获取第一次结果
url_post_first = "http://58.213.147.230:7001/Jsjzyxyglpt/faces/public/projectList.jsp"

In [10]:
post_data_first = {"projectChildSelectedTabPageIndex": '1',
"projectWinSelectedTabPageIndex" : '1',
"basicWinSelectedTabPageIndex" : '1',
"peopleWinSelectedTabPageIndex" : '1',
"form:refreshAct" : '',
"form:page" : '1',
"form:_id0" : 'builderLicence',
"form:_id2" : '',
"form:_id3" : '',
"form:_id4" : '',
"form:_id5" : '3206',
"form:_id6" : '',
"form:_id7" : '',
"form:checkCode" : str_checkCode,
"com.sun.faces.VIEW" : post_com_sun_faces_VIEW,
"form" : "form"
}

In [11]:
resp_post_first = reqs.post(url_post_first, post_data_first, headers=headers, proxies=proxies)

In [12]:
resp_html = resp_post_first.content

In [13]:
# 获取总页码数
soup_post_first = BeautifulSoup(resp_post_first.content.decode(encoding='gbk'),"html5lib")

all_pages = int(soup_post_first.form.find_all('table')[3].tbody.tr.td.div.find_all('div')[-2].table.tbody.tr.td.div.table.tbody.tr.find_all('td')[5].b.text)

In [14]:
print(all_pages)

319


In [15]:
pd_Data = pd.DataFrame(columns=['序号','许可证号','工程名称','发证日期','建设单位','施工单位','项目经理','监理单位','项目总监','勘查单位','设计单位','项目所在地','合同价(万元)','建设规模'])
def get_parse_rlt(html, pd_Data):
    soup_html = BeautifulSoup(html, "html5lib")
    rlt = dict()
    for page_project_num in range(30):
        # /html/body/form/table/tbody/tr/td/div[1]/div[3]/table/tbody/tr[1]/td[2]/div/nobr
        rlt['序号'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[0].div.text
        # 许可证号
        rlt['许可证号'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[1].div.nobr.text
        # 工程名称
        #/html/body/form/table/tbody/tr/td/div[1]/div[3]/table/tbody/tr[1]/td[3]/div/nobr
        rlt['工程名称'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[2].div.nobr.text
        rlt['发证日期'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[3].div.nobr.text
        rlt['建设单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[4].div.nobr.text
        rlt['施工单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[5].div.nobr.text
        rlt['项目经理'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[6].div.nobr.text
        rlt['监理单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[7].div.nobr.text
        rlt['项目总监'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[8].div.nobr.text
        rlt['勘查单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[9].div.nobr.text
        rlt['设计单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[10].div.nobr.text
        rlt['项目所在地'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[11].div.nobr.text
        rlt['合同价(万元)'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[12].div.nobr.text
        rlt['建设规模'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[13].div.nobr.text
        pd_Data=pd_Data.append(pd.DataFrame(rlt,index=[rlt['序号']]))
    print('page_project_num:',page_project_num, 'len(pd_Data):',len(pd_Data))
    return pd_Data

In [16]:
pd_Data = get_parse_rlt(resp_post_first.content.decode(encoding='gbk'), pd_Data)

page_project_num: 29 len(pd_Data): 30


In [17]:
# 获取后页内容
# 构造post_data
post_data = post_data_first.copy()
post_data.pop('form:refreshAct')
post_data['form:checkCode'] = ''

''

In [18]:
for page_num in range(2, 4+1):
#for page_num in range(2, all_pages+1):
    time.sleep(15)
    post_data['form:page'] = page_num
    resp_post = reqs.post(url_post_first, post_data, headers=headers, proxies=proxies)
    print(page_num,len(resp_post.content.decode(encoding='gbk',errors='ignore')))
    pd_Data = get_parse_rlt(resp_post.content.decode(encoding='gbk',errors='ignore'),pd_Data)

2 195004
page_project_num: 29 len(pd_Data): 60
3 195098
page_project_num: 29 len(pd_Data): 90
4 195044
page_project_num: 29 len(pd_Data): 120


In [19]:
pd_Data.to_excel("建筑施工许可20210112.xlsx", index=False)

In [ ]:
#解析内容文件

In [25]:
html = resp_post.content.decode(encoding='gbk',)

In [40]:
def get_parse_rlt(html, pd_Data):
    soup_html = BeautifulSoup(html, "html5lib")
    rlt = dict()
    for page_project_num in range(30):
        # /html/body/form/table/tbody/tr/td/div[1]/div[3]/table/tbody/tr[1]/td[2]/div/nobr
        rlt['序号'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[0].div.text
        # 许可证号
        rlt['许可证号'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[1].div.nobr.text
        # 工程名称
        #/html/body/form/table/tbody/tr/td/div[1]/div[3]/table/tbody/tr[1]/td[3]/div/nobr
        rlt['工程名称'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[2].div.nobr.text
        rlt['发证日期'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[3].div.nobr.text
        rlt['建设单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[4].div.nobr.text
        rlt['施工单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[5].div.nobr.text
        rlt['项目经理'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[6].div.nobr.text
        rlt['监理单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[7].div.nobr.text
        rlt['项目总监'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[8].div.nobr.text
        rlt['勘查单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[9].div.nobr.text
        rlt['设计单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[10].div.nobr.text
        rlt['项目所在地'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[11].div.nobr.text
        rlt['合同价(万元)'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[12].div.nobr.text
        rlt['建设规模'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[13].div.nobr.text
        pd_Data=pd_Data.append(pd.DataFrame(rlt,index=[rlt['序号']]))
    print(page_project_num,len(pd_Data))
    return pd_Data

In [28]:
soup_html = BeautifulSoup(html,"html5lib")
rlt = dict()
for page_project_num in range(30):
    # /html/body/form/table/tbody/tr/td/div[1]/div[3]/table/tbody/tr[1]/td[2]/div/nobr
    rlt['序号'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[0].div.text
    # 许可证号
    rlt['许可证号'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[1].div.nobr.text
    # 工程名称
    #/html/body/form/table/tbody/tr/td/div[1]/div[3]/table/tbody/tr[1]/td[3]/div/nobr
    rlt['工程名称'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[2].div.nobr.text
    rlt['发证日期'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[3].div.nobr.text
    rlt['建设单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[4].div.nobr.text
    rlt['施工单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[5].div.nobr.text
    rlt['项目经理'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[6].div.nobr.text
    rlt['监理单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[7].div.nobr.text
    rlt['项目总监'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[8].div.nobr.text
    rlt['勘查单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[9].div.nobr.text
    rlt['设计单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[10].div.nobr.text
    rlt['项目所在地'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[11].div.nobr.text
    rlt['合同价(万元)'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[12].div.nobr.text
    rlt['建设规模'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[page_project_num].find_all('td')[13].div.nobr.text
    pd_Data=pd_Data.append(pd.DataFrame(rlt,index=[rlt['序号']]))

In [20]:
pd_Data

,序号,许可证号,工程名称,发证日期,建设单位,施工单位,项目经理,监理单位,项目总监,勘查单位,设计单位,项目所在地,合同价(万元),建设规模
1,1,320691202101120101,年产7.74万吨工业加工介质、高分子新材料和有机锡催化剂项目：乙类仓库、丙类仓库,2021-01-12,江苏鑫露化工新材料有限公司,南通晶硕建设工程有限公司,陈月江,南通方圆工程监理咨询有限公司,肖剑雄,江苏苏州地质工程勘察院,南通中房建筑设计研究院有限公司,南通市经济技术开发区,1460.0000,9979.22平方米
2,2,320621202101110401,泰康居大楼1,2021-01-11,海安县曲塘镇敬老院,南通达洲建设工程有限公司,仲陈剑,江苏佰鸿工程项目管理有限公司,刘笑宇,江苏苏州地质工程勘察院,南京典筑建筑设计有限公司,南通市海安县,1105.6600,3989.7平方米
3,3,320621202101110301,尊元里1~6#楼及地下车库（含人防）,2021-01-11,海安欣瑞置业有限公司,南通华汇建筑工程有限公司,夏建华,泰州致宸建设工程监理有限公司,许秀云,江苏科泰岩土工程有限公司,上海华东建设发展设计有限公司,南通市海安县,11865.1100,66420.48平方米
4,4,320681202101110301,1#厂房、2#办公楼、3#厂房、4#门卫室,2021-01-11,红通（启东）机械装备有限公司,江苏镇淮建设集团有限公司,王昌永,南通瑞达建设监理有限公司,王能飞,启东市建筑设计院有限公司,上海亚新工程顾问有限公司,南通市启东市,22000.0000,53258.69平方米
5,5,320681202101110201,车间三,2021-01-11,江苏泰林建设有限公司,上海真通建筑安装工程有限公司,张宇超,无,无,南通勘察设计有限公司,启东市第二建筑设计院有限公司,南通市启东市,95.0000,412.03平方米
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,116,320602202012180201,工农路东、洪江路北R18044地块二期5#楼公共区域精装修,2020-12-18,南通博恒置业有限公司,南通八建集团有限公司,陈超,江苏润扬项目管理有限公司,肖志平,无,江苏浩森建筑设计有限公司,南通市崇川区,28.3400,0.00平方米
117,117,320602202012180101,工农路东、洪江路北R18044地块一期1#-4#楼、6#-9#楼公共区域精装修,2020-12-18,南通博恒置业有限公司,南通八建集团有限公司,陈超,江苏润扬项目管理有限公司,肖志平,无,江苏浩森建筑设计有限公司,南通市崇川区,188.0600,0.00平方米
118,118,320621202012170199,海安石油公司三丰加油站（站房、罩棚）,2020-12-17,中国石化销售股份有限公司江苏南通海安石油分公司,华东建设安装有限公司,赵志锋,江苏翔宇工程监理咨询有限公司,席俊君,南通天利岩土工程勘察有限公司,南通市港闸建筑设计院有限公司,南通市海安县,327.3200,810.70平方米
119,119,320612202012170101,车间一、车间二、附属用房、门卫消防室配电间泵房及地下消防水池,2020-12-17,南通市通州区拓玉工艺品有限公司,南通知行建设工程有限公司,王亮亮,无,无,江苏科信岩土工程勘察有限公司,中铁华铁工程设计集团有限公司,南通市通州区,1880.0000,建筑面积13796.36平方米


In [90]:
# /html/body/form/table/tbody/tr/td/div[1]/div[3]/table/tbody/tr[1]/td[2]/div/nobr
rlt['序号'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[0].find_all('td')[0].div.text
# 许可证号
rlt['许可证号'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[0].find_all('td')[1].div.nobr.text
# 工程名称
#/html/body/form/table/tbody/tr/td/div[1]/div[3]/table/tbody/tr[1]/td[3]/div/nobr
rlt['工程名称'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[0].find_all('td')[2].div.nobr.text
rlt['发证日期'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[0].find_all('td')[3].div.nobr.text
rlt['建设单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[0].find_all('td')[4].div.nobr.text
rlt['施工单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[0].find_all('td')[5].div.nobr.text
rlt['项目经理'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[0].find_all('td')[6].div.nobr.text
rlt['监理单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[0].find_all('td')[7].div.nobr.text
rlt['项目总监'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[0].find_all('td')[8].div.nobr.text
rlt['勘查单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[0].find_all('td')[9].div.nobr.text
rlt['设计单位'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[0].find_all('td')[10].div.nobr.text
rlt['项目所在地'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[0].find_all('td')[11].div.nobr.text
rlt['合同价(万元)'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[0].find_all('td')[12].div.nobr.text
rlt['建设规模'] = soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[0].find_all('td')[13].div.nobr.text
pd_rlt=pd_Data.append(pd.DataFrame(rlt,index=[rlt['序号']]))

In [91]:
rlt

{'序号': '91',
 '许可证号': '320681202012150301',
 '工程名称': '教学楼、配电房、门卫、消控',
 '发证日期': '2020-12-15',
 '建设单位': '启东市宝宝幼儿园',
 '施工单位': '江苏银洲建设集团有限公司',
 '项目经理': '杨桂兰',
 '监理单位': '南通瑞达建设监理有限公司',
 '项目总监': '柏晓花',
 '勘查单位': '南通勘察设计有限公司',
 '设计单位': '启东市第二建筑设计院有限公司',
 '项目所在地': '南通市启东市',
 '合同价(万元)': '1264.4600',
 '建设规模': '5452.81平方米'}

In [69]:
# 许可证号
# /html/body/form/table/tbody/tr/td/div[1]/div[3]/table/tbody/tr[2]/td[2]/div/nobr
soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')[5].tbody.find_all('tr')[2].find_all('td')[1].div.nobr.text

'320681202012150101'

In [ ]:
# #SimpleRichTableBody33d558ce8f1049a0990bda953d9157cc > table > tbody > tr:nth-child(1) > td:nth-child(2) > div > nobr
# #SimpleRichTableBody84eb6df1b2cf480eb31c39a9f25c024e > table > tbody > tr:nth-child(1) > td:nth-child(2) > div > nobr
# //*[@id="SimpleRichTableBody84eb6df1b2cf480eb31c39a9f25c024e"]/table/tbody/tr[1]/td[2]/div/nobr


In [41]:
pd_Data = pd.DataFrame(columns=['序号','许可证号','工程名称','发证日期','建设单位','施工单位','项目经理','监理单位','项目总监','勘查单位','设计单位','项目所在地','合同价(万元)','建设规模'])

In [99]:
pd.DataFrame(rlt,index=[91])

,序号,许可证号,工程名称,发证日期,建设单位,施工单位,项目经理,监理单位,项目总监,勘查单位,设计单位,项目所在地,合同价(万元),建设规模
91,91,320681202012150301,教学楼、配电房、门卫、消控,2020-12-15,启东市宝宝幼儿园,江苏银洲建设集团有限公司,杨桂兰,南通瑞达建设监理有限公司,柏晓花,南通勘察设计有限公司,启东市第二建筑设计院有限公司,南通市启东市,1264.4600,5452.81平方米


In [106]:
pd_rlt=pd_Data.append(pd.DataFrame(rlt,index=[91]))

In [108]:
pd.DataFrame(rlt,index=[rlt['序号']])

,序号,许可证号,工程名称,发证日期,建设单位,施工单位,项目经理,监理单位,项目总监,勘查单位,设计单位,项目所在地,合同价(万元),建设规模
91,91,320681202012150301,教学楼、配电房、门卫、消控,2020-12-15,启东市宝宝幼儿园,江苏银洲建设集团有限公司,杨桂兰,南通瑞达建设监理有限公司,柏晓花,南通勘察设计有限公司,启东市第二建筑设计院有限公司,南通市启东市,1264.4600,5452.81平方米


In [107]:
pd_rlt

,序号,许可证号,工程名称,发证日期,建设单位,施工单位,项目经理,监理单位,项目总监,勘查单位,设计单位,项目所在地,合同价(万元),建设规模
91,91,320681202012150301,教学楼、配电房、门卫、消控,2020-12-15,启东市宝宝幼儿园,江苏银洲建设集团有限公司,杨桂兰,南通瑞达建设监理有限公司,柏晓花,南通勘察设计有限公司,启东市第二建筑设计院有限公司,南通市启东市,1264.4600,5452.81平方米


In [52]:
# 页码table
for i,soup_obj in enumerate(soup_html.form.find_all('table')[3].tbody.tr.td.div.find_all('table')):
    if soup_obj.find_all(text='320681202012150301') != []:
        print(i,soup_obj.find_all(text='320681202012150301'))

5 ['320681202012150301']


In [69]:
# 结果table
soup_post_first.form.find_all('table')[3].tbody.tr.td.div.find_all('div')[468].table.tbody.tr.td.div.table.tbody.tr.find_all('td')[5].b.text

'317'

In [ ]:
#form > table > tbody > tr > td > div:nth-child(2) > div:nth-child(4) > table > tbody > tr > td > div > table > tbody > tr > td:nth-child(6) > b:nth-child(1)
#document.querySelector("#form > table > tbody > tr > td > div:nth-child(2) > div:nth-child(4) > table > tbody > tr > td > div > table > tbody > tr > td:nth-child(6) > b:nth-child(1)")
#//*[@id="form"]/table/tbody/tr/td/div[1]/div[4]/table/tbody/tr/td/div/table/tbody/tr/td[6]/b[1]

In [73]:
data=urlencode(post_data)

In [74]:
data

'projectChildSelectedTabPageIndex=1&projectWinSelectedTabPageIndex=1&basicWinSelectedTabPageIndex=1&peopleWinSelectedTabPageIndex=1&form%3ArefreshAct=&form%3Apage=2&form%3A_id0=builderLicence&form%3A_id2=&form%3A_id3=&form%3A_id4=&form%3A_id5=3206&form%3A_id6=&form%3A_id7=&form%3AcheckCode=&com.sun.faces.VIEW=rO0ABXNyACBjb20uc3VuLmZhY2VzLnV0aWwuVHJlZVN0cnVjdHVyZRRmG0QclWAgAgAETAAIY2hpbGRyZW50ABVMamF2YS91dGlsL0FycmF5TGlzdDtMAAljbGFzc05hbWV0ABJMamF2YS9sYW5nL1N0cmluZztMAAZmYWNldHN0ABNMamF2YS91dGlsL0hhc2hNYXA7TAACaWRxAH4AAnhwc3IAE2phdmEudXRpbC5BcnJheUxpc3R4gdIdmcdhnQMAAUkABHNpemV4cAAAAAF3BAAAAApzcQB%2BAABzcQB%2BAAUAAAAKdwQAAAAKc3EAfgAAcHQALGphdmF4LmZhY2VzLmNvbXBvbmVudC5odG1sLkh0bWxDb21tYW5kQnV0dG9ucHQACnJlZnJlc2hBY3RzcQB%2BAABwdAAqamF2YXguZmFjZXMuY29tcG9uZW50Lmh0bWwuSHRtbElucHV0SGlkZGVucHQABHBhZ2VzcQB%2BAABzcQB%2BAAUAAAABdwQAAAAKc3EAfgAAcHQAI2phdmF4LmZhY2VzLmNvbXBvbmVudC5VSVNlbGVjdEl0ZW1zcHQABF9pZDF4dAAsamF2YXguZmFjZXMuY29tcG9uZW50Lmh0bWwuSHRtbFNlbGVjdE9uZU1lbnVwdAAEX2lkMHNxAH4AAHB0AChqYXZ

In [75]:
html_task = reqs.post(url, data, headers=headers, proxies=proxies)

In [76]:
# 2nd page
post_data = {"projectChildSelectedTabPageIndex": '1',
"projectWinSelectedTabPageIndex" : '1',
"basicWinSelectedTabPageIndex" : '1',
"peopleWinSelectedTabPageIndex" : '1',
"form:page" : '2',
"form:_id0" : 'builderLicence',
"form:_id2" : '',
"form:_id3" : '',
"form:_id4" : '',
"form:_id5" : '3206',
"form:_id6" : '',
"form:_id7" : '',
"form:checkCode" : '',
"com.sun.faces.VIEW" : "rO0ABXNyACBjb20uc3VuLmZhY2VzLnV0aWwuVHJlZVN0cnVjdHVyZRRmG0QclWAgAgAETAAIY2hpbGRyZW50ABVMamF2YS91dGlsL0FycmF5TGlzdDtMAAljbGFzc05hbWV0ABJMamF2YS9sYW5nL1N0cmluZztMAAZmYWNldHN0ABNMamF2YS91dGlsL0hhc2hNYXA7TAACaWRxAH4AAnhwc3IAE2phdmEudXRpbC5BcnJheUxpc3R4gdIdmcdhnQMAAUkABHNpemV4cAAAAAF3BAAAAApzcQB+AABzcQB+AAUAAAAKdwQAAAAKc3EAfgAAcHQALGphdmF4LmZhY2VzLmNvbXBvbmVudC5odG1sLkh0bWxDb21tYW5kQnV0dG9ucHQACnJlZnJlc2hBY3RzcQB+AABwdAAqamF2YXguZmFjZXMuY29tcG9uZW50Lmh0bWwuSHRtbElucHV0SGlkZGVucHQABHBhZ2VzcQB+AABzcQB+AAUAAAABdwQAAAAKc3EAfgAAcHQAI2phdmF4LmZhY2VzLmNvbXBvbmVudC5VSVNlbGVjdEl0ZW1zcHQABF9pZDF4dAAsamF2YXguZmFjZXMuY29tcG9uZW50Lmh0bWwuSHRtbFNlbGVjdE9uZU1lbnVwdAAEX2lkMHNxAH4AAHB0AChqYXZheC5mYWNlcy5jb21wb25lbnQuaHRtbC5IdG1sSW5wdXRUZXh0cHQABF9pZDJzcQB+AABwcQB+ABdwdAAEX2lkM3NxAH4AAHBxAH4AF3B0AARfaWQ0c3EAfgAAcHEAfgAXcHQABF9pZDVzcQB+AABwcQB+ABdwdAAEX2lkNnNxAH4AAHBxAH4AF3B0AARfaWQ3c3EAfgAAcHEAfgAXcHQACWNoZWNrQ29kZXh0ACNqYXZheC5mYWNlcy5jb21wb25lbnQuaHRtbC5IdG1sRm9ybXB0AARmb3JteHQAKG9yZy5hamF4NGpzZi5mcmFtZXdvcmsuYWpheC5BamF4Vmlld1Jvb3RwdAAJX3ZpZXdSb290dXIAE1tMamF2YS5sYW5nLk9iamVjdDuQzlifEHMpbAIAAHhwAAAAAnVxAH4AKQAAAAJ1cQB+ACkAAAAEdXEAfgApAAAACHNyABFqYXZhLnV0aWwuSGFzaE1hcAUH2sHDFmDRAwACRgAKbG9hZEZhY3RvckkACXRocmVzaG9sZHhwP0AAAAAAAAx3CAAAABAAAAABdAAkamF2YXguZmFjZXMud2ViYXBwLkNVUlJFTlRfVklFV19ST09UcQB+ADB4cHQACV92aWV3Um9vdHEAfgAoc3IAEWphdmEubGFuZy5Cb29sZWFuzSBygNWc+u4CAAFaAAV2YWx1ZXhwAXNxAH4AMgBwcHQACkhUTUxfQkFTSUN0ABcvcHVibGljL3Byb2plY3RMaXN0LmpzcHNyABBqYXZhLnV0aWwuTG9jYWxlfvgRYJww+ewCAARJAAhoYXNoY29kZUwAB2NvdW50cnlxAH4AAkwACGxhbmd1YWdlcQB+AAJMAAd2YXJpYW50cQB+AAJ4cP////90AAJDTnQAAnpodAAAdXEAfgApAAAABXEAfgA0cQB+ADRwcQB+ADRxAH4ANHVxAH4AKQAAAAF1cQB+ACkAAAACdXEAfgApAAAAFnVxAH4AKQAAAAhzcQB+AC4/QAAAAAAADHcIAAAAEAAAAAF0ACBqYXZheC5mYWNlcy53ZWJhcHAuQ09NUE9ORU5UX0lEU3NxAH4ABQAAAAp3BAAAAApxAH4AC3EAfgAOcQB+ABVxAH4AGHEAfgAacQB+ABxxAH4AHnEAfgAgcQB+ACJxAH4AJHh4cHQABGZvcm1xAH4AJnEAfgAzcQB+ADR0ABBqYXZheC5mYWNlcy5Gb3JtcHBwcHQAIWFwcGxpY2F0aW9uL3gtd3d3LWZvcm0tdXJsZW5jb2RlZHBwcHBwcHBwcHBwcHBwcHBwdXEAfgApAAAACnVxAH4AKQAAAAJ1cQB+ACkAAAAddXEAfgApAAAABnVxAH4AKQAAAAhzcQB+AC4/QAAAAAAADHcIAAAAEAAAAAB4cHQAD2Zvcm06cmVmcmVzaEFjdHEAfgALcQB+ADNxAH4ANHQAEmphdmF4LmZhY2VzLkJ1dHRvbnBzcgAmamF2YXguZmFjZXMuY29tcG9uZW50LlN0YXRlSG9sZGVyU2F2ZXJZyrM9k5zNTQIAAkwACWNsYXNzTmFtZXEAfgACTAAKc2F2ZWRTdGF0ZXQAEkxqYXZhL2xhbmcvT2JqZWN0O3hwdAAiY29tLnN1bi5mYWNlcy5lbC5NZXRob2RCaW5kaW5nSW1wbHVxAH4AKQAAAAR0AAdyZWZyZXNoc3EAfgBPdAAhY29tLnN1bi5mYWNlcy5lbC5WYWx1ZUJpbmRpbmdJbXBsdAAPSW5mb3JtYXRpb25CZWFucHQAGiN7SW5mb3JtYXRpb25CZWFuLnJlZnJlc2h9cHEAfgA0cQB+ADRwcHBwcQB+ADRxAH4ANHBwcHBwcHBwcHBwcHBwcHBxAH4ANHEAfgA0dAANZGlzcGxheTpub25lO3BwcHQABnN1Ym1pdHVxAH4AKQAAAAB1cQB+ACkAAAACdXEAfgApAAAAAXVxAH4AKQAAAAp1cQB+ACkAAAADdXEAfgApAAAACHNxAH4ALj9AAAAAAAAMdwgAAAAQAAAAAHh1cQB+ACkAAAACdXIAE1tMamF2YS5sYW5nLlN0cmluZzut0lbn6R17RwIAAHhwAAAAAXQABXZhbHVldXEAfgApAAAAAXNxAH4AT3EAfgBWdAAwSW5mb3JtYXRpb25CZWFuLnNwbGl0UGFnZURhdGFQcm9qZWN0LmN1cnJlbnRQYWdldAAJZm9ybTpwYWdlcQB+AA5xAH4AM3EAfgA0dAASamF2YXguZmFjZXMuSGlkZGVucHBwcQB+ADRxAH4ANHEAfgA0cQB+ADNxAH4ANHEAfgA0cHBwdXEAfgApAAAAAHVxAH4AKQAAAAJ1cQB+ACkAAAAcdXEAfgApAAAACnVxAH4AKQAAAAN1cQB+ACkAAAAIc3EAfgAuP0AAAAAAAAx3CAAAABAAAAABcQB+AEJzcQB+AAUAAAABdwQAAAAKcQB+ABN4eHVxAH4AKQAAAAJ1cQB+AGMAAAABcQB+AGV1cQB+ACkAAAABc3EAfgBPcQB+AFZ0ACRJbmZvcm1hdGlvbkJlYW4uZmlsdGVyUHJvamVjdC5maWVsZDF0AAlmb3JtOl9pZDBxAH4AFXEAfgAzcQB+ADR0ABBqYXZheC5mYWNlcy5NZW51cHBwcQB+ADRxAH4ANHEAfgA0cQB+ADNxAH4ANHEAfgA0cHBwcHBxAH4ANHEAfgA0cHBwcHQAPHNlbGYubG9jYXRpb249J3Byb2plY3RMaXN0LmpzcD9wcm9qZWN0Tm9kZVR5cGU9Jyt0aGlzLnZhbHVlO3BwcHBwcHBwcHBwcHEAfgA0cQB+ADR0AAl3aWR0aDoxMDBwcHB1cQB+ACkAAAABdXEAfgApAAAAAnVxAH4AKQAAAAJ1cQB+ACkAAAAIc3EAfgAuP0AAAAAAAAx3CAAAABAAAAAAeHVxAH4AKQAAAAJ1cQB+AGMAAAABcQB+AGV1cQB+ACkAAAABc3EAfgBPcQB+AFZ0ACpDb2RlVHJhbnNsYXRvckJlYW4uc2VsZWN0SXRlbXMucHJvamVjdE5vZGV0AAlmb3JtOl9pZDFxAH4AE3EAfgAzcQB+ADRwcHB1cQB+ACkAAAAAdXEAfgApAAAAAnVxAH4AKQAAAB91cQB+ACkAAAAKdXEAfgApAAAAA3VxAH4AKQAAAAhzcQB+AC4/QAAAAAAADHcIAAAAEAAAAAB4dXEAfgApAAAAAnVxAH4AYwAAAAFxAH4AZXVxAH4AKQAAAAFzcQB+AE9xAH4AVnQAJEluZm9ybWF0aW9uQmVhbi5maWx0ZXJQcm9qZWN0LmZpZWxkMnQACWZvcm06X2lkMnEAfgAYcQB+ADNxAH4ANHQAEGphdmF4LmZhY2VzLlRleHRwcHBxAH4ANHEAfgA0cQB+ADRxAH4AM3EAfgA0cQB+ADRwcHBwcHBxAH4ANHEAfgA0cHNyABFqYXZhLmxhbmcuSW50ZWdlchLioKT3gYc4AgABSQAFdmFsdWV4cgAQamF2YS5sYW5nLk51bWJlcoaslR0LlOCLAgAAeHCAAAAAcQB+ADRwcHBwcHBwcHBwcHBwcHEAfgA0cQB+ADRzcQB+AJWAAAAAcQB+ADR0AAp3aWR0aDoyMzA7cHBwdXEAfgApAAAAAHVxAH4AKQAAAAJ1cQB+ACkAAAAfdXEAfgApAAAACnVxAH4AKQAAAAN1cQB+ACkAAAAIc3EAfgAuP0AAAAAAAAx3CAAAABAAAAAAeHVxAH4AKQAAAAJ1cQB+AGMAAAABcQB+AGV1cQB+ACkAAAABc3EAfgBPcQB+AFZ0ACRJbmZvcm1hdGlvbkJlYW4uZmlsdGVyUHJvamVjdC5maWVsZDN0AAlmb3JtOl9pZDNxAH4AGnEAfgAzcQB+ADRxAH4AlHBwcHEAfgA0cQB+ADRxAH4ANHEAfgAzcQB+ADRxAH4ANHBwcHBwcHEAfgA0cQB+ADRwc3EAfgCVgAAAAHEAfgA0cHBwcHBwcHBwcHBwcHBxAH4ANHEAfgA0c3EAfgCVgAAAAHEAfgA0dAAKd2lkdGg6MTUwO3BwcHVxAH4AKQAAAAB1cQB+ACkAAAACdXEAfgApAAAAH3VxAH4AKQAAAAp1cQB+ACkAAAADdXEAfgApAAAACHNxAH4ALj9AAAAAAAAMdwgAAAAQAAAAAHh1cQB+ACkAAAACdXEAfgBjAAAAAXEAfgBldXEAfgApAAAAAXNxAH4AT3EAfgBWdAAkSW5mb3JtYXRpb25CZWFuLmZpbHRlclByb2plY3QuZmllbGQ0dAAJZm9ybTpfaWQ0cQB+ABxxAH4AM3EAfgA0cQB+AJRwcHBxAH4ANHEAfgA0cQB+ADRxAH4AM3EAfgA0cQB+ADRwcHBwcHBxAH4ANHEAfgA0cHNxAH4AlYAAAABxAH4ANHBwcHBwcHBwcHBwcHBwcQB+ADRxAH4ANHNxAH4AlYAAAABxAH4ANHQACndpZHRoOjE4MDtwcHB1cQB+ACkAAAAAdXEAfgApAAAAAnVxAH4AKQAAAB91cQB+ACkAAAAKdXEAfgApAAAAA3VxAH4AKQAAAAhzcQB+AC4/QAAAAAAADHcIAAAAEAAAAAB4dXEAfgApAAAAAnVxAH4AYwAAAAFxAH4AZXVxAH4AKQAAAAFzcQB+AE9xAH4AVnQAJEluZm9ybWF0aW9uQmVhbi5maWx0ZXJQcm9qZWN0LmZpZWxkNXQACWZvcm06X2lkNXEAfgAecQB+ADNxAH4ANHEAfgCUcHBwcQB+ADRxAH4ANHEAfgA0cQB+ADNxAH4ANHEAfgA0cHBwcHBwcQB+ADRxAH4ANHBzcQB+AJWAAAAAcQB+ADRwcHBwcHBwcHBwcHBwcHEAfgA0cQB+ADRzcQB+AJWAAAAAcQB+ADRxAH4AqXBwcHVxAH4AKQAAAAB1cQB+ACkAAAACdXEAfgApAAAAH3VxAH4AKQAAAAp1cQB+ACkAAAADdXEAfgApAAAACHNxAH4ALj9AAAAAAAAMdwgAAAAQAAAAAHh1cQB+ACkAAAACdXEAfgBjAAAAAXEAfgBldXEAfgApAAAAAXNxAH4AT3EAfgBWdAAkSW5mb3JtYXRpb25CZWFuLmZpbHRlclByb2plY3QuZmllbGQ2dAAJZm9ybTpfaWQ2cQB+ACBxAH4AM3EAfgA0cQB+AJRwcHBxAH4ANHEAfgA0cQB+ADRxAH4AM3EAfgA0cQB+ADRwcHBwcHBxAH4ANHEAfgA0cHNxAH4AlYAAAABxAH4ANHBwcHBwcHBwcHBwcHBwcQB+ADRxAH4ANHNxAH4AlYAAAABxAH4ANHQACndpZHRoOjIwMDtwcHB1cQB+ACkAAAAAdXEAfgApAAAAAnVxAH4AKQAAAB91cQB+ACkAAAAKdXEAfgApAAAAA3VxAH4AKQAAAAhzcQB+AC4/QAAAAAAADHcIAAAAEAAAAAB4dXEAfgApAAAAAnVxAH4AYwAAAAFxAH4AZXVxAH4AKQAAAAFzcQB+AE9xAH4AVnQAJEluZm9ybWF0aW9uQmVhbi5maWx0ZXJQcm9qZWN0LmZpZWxkN3QACWZvcm06X2lkN3EAfgAicQB+ADNxAH4ANHEAfgCUcHBwcQB+ADRxAH4ANHEAfgA0cQB+ADNxAH4ANHEAfgA0cHBwcHBwcQB+ADRxAH4ANHBzcQB+AJWAAAAAcQB+ADRwcHBwcHBwcHBwcHBwcHEAfgA0cQB+ADRzcQB+AJWAAAAAcQB+ADRxAH4A2HBwcHVxAH4AKQAAAAB1cQB+ACkAAAACdXEAfgApAAAAH3VxAH4AKQAAAAp1cQB+ACkAAAADdXEAfgApAAAACHNxAH4ALj9AAAAAAAAMdwgAAAAQAAAAAHh1cQB+ACkAAAACdXEAfgBjAAAAAXEAfgBldXEAfgApAAAAAXNxAH4AT3EAfgBWdAAZSW5mb3JtYXRpb25CZWFuLmNoZWNrQ29kZXQADmZvcm06Y2hlY2tDb2RlcQB+ACRxAH4AM3EAfgA0cQB+AJRwcHBxAH4ANHEAfgA0cQB+ADRxAH4AM3EAfgA0cQB+ADRwcHBwcHBxAH4ANHEAfgA0cHNxAH4AlYAAAABxAH4ANHBwcHBwcHBwcHBwcHBwcQB+ADRxAH4ANHNxAH4AlYAAAABxAH4ANHQACndpZHRoOjEwMDtwcHB1cQB+ACkAAAAA",
"form" : "form"
}

In [78]:
html_task = reqs.post(url, post_data, headers=headers, proxies=proxies)

In [79]:
print(html_task.content.decode(encoding='gbk'))














<html>
<head>
    <title>江苏省建筑市场监管与诚信信息一体化平台</title>
 	<meta http-equiv="X-UA-Compatible" content="IE=EmulateIE8" >
<meta http-equiv="content-type" content="text/html;charset=GBK">
<meta http-equiv="pragma" content="no-cache">
<meta http-equiv="cache-control" content="no-cache">
<meta http-equiv="expires" content="0">
<script charset="GBK" type='text/javascript' src='/Jsjzyxyglpt/temp/script/jquery.js'></script>
<script charset="GBK" type='text/javascript' src='/Jsjzyxyglpt/temp/script/alphaClassManager.js'></script>
<script charset="GBK" type='text/javascript' src='/Jsjzyxyglpt/temp/script/commonFunction.js'></script>
<script charset="GBK" type='text/javascript' src='/Jsjzyxyglpt/temp/script/animationEffects.js'></script>
<script charset="GBK" type='text/javascript' src='/Jsjzyxyglpt/temp/script/ajax.js'></script>
<script charset="GBK" type='text/javascript' src='/Jsjzyxyglpt/temp/script/dataTable.js'></script>
<script charset="GBK" type='text/javascript' src='/Jsjzyxyglp

In [31]:
with open('test.html', 'wb') as f:
    f.write(resp_post.content)

202143

In [ ]:
print(reqs.cookies.get_dict())

In [ ]:
text = req.content

In [ ]:
text.decode(encoding='gbk')

In [81]:
a={1:2,3:4}

In [86]:
soup = BeautifulSoup(html_task.content.decode(encoding='gbk'), "lxml")

In [89]:
soup.prettify

<bound method Tag.prettify of <html>
<head>
<title>江苏省建筑市场监管与诚信信息一体化平台</title>
<meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible"/>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<meta content="no-cache" http-equiv="pragma"/>
<meta content="no-cache" http-equiv="cache-control"/>
<meta content="0" http-equiv="expires"/>
<script charset="GBK" src="/Jsjzyxyglpt/temp/script/jquery.js" type="text/javascript"></script>
<script charset="GBK" src="/Jsjzyxyglpt/temp/script/alphaClassManager.js" type="text/javascript"></script>
<script charset="GBK" src="/Jsjzyxyglpt/temp/script/commonFunction.js" type="text/javascript"></script>
<script charset="GBK" src="/Jsjzyxyglpt/temp/script/animationEffects.js" type="text/javascript"></script>
<script charset="GBK" src="/Jsjzyxyglpt/temp/script/ajax.js" type="text/javascript"></script>
<script charset="GBK" src="/Jsjzyxyglpt/temp/script/dataTable.js" type="text/javascript"></script>
<script charset="GBK" src="/Jsjzyxyglpt/temp

In [107]:
soup.find_all(attrs={'name':'com.sun.faces.VIEW'})[0].attrs['value']

'rO0ABXNyACBjb20uc3VuLmZhY2VzLnV0aWwuVHJlZVN0cnVjdHVyZRRmG0QclWAgAgAETAAIY2hpbGRyZW50ABVMamF2YS91dGlsL0FycmF5TGlzdDtMAAljbGFzc05hbWV0ABJMamF2YS9sYW5nL1N0cmluZztMAAZmYWNldHN0ABNMamF2YS91dGlsL0hhc2hNYXA7TAACaWRxAH4AAnhwc3IAE2phdmEudXRpbC5BcnJheUxpc3R4gdIdmcdhnQMAAUkABHNpemV4cAAAAAF3BAAAAApzcQB+AABzcQB+AAUAAAAKdwQAAAAKc3EAfgAAcHQALGphdmF4LmZhY2VzLmNvbXBvbmVudC5odG1sLkh0bWxDb21tYW5kQnV0dG9ucHQACnJlZnJlc2hBY3RzcQB+AABwdAAqamF2YXguZmFjZXMuY29tcG9uZW50Lmh0bWwuSHRtbElucHV0SGlkZGVucHQABHBhZ2VzcQB+AABzcQB+AAUAAAABdwQAAAAKc3EAfgAAcHQAI2phdmF4LmZhY2VzLmNvbXBvbmVudC5VSVNlbGVjdEl0ZW1zcHQABF9pZDF4dAAsamF2YXguZmFjZXMuY29tcG9uZW50Lmh0bWwuSHRtbFNlbGVjdE9uZU1lbnVwdAAEX2lkMHNxAH4AAHB0AChqYXZheC5mYWNlcy5jb21wb25lbnQuaHRtbC5IdG1sSW5wdXRUZXh0cHQABF9pZDJzcQB+AABwcQB+ABdwdAAEX2lkM3NxAH4AAHBxAH4AF3B0AARfaWQ0c3EAfgAAcHEAfgAXcHQABF9pZDVzcQB+AABwcQB+ABdwdAAEX2lkNnNxAH4AAHBxAH4AF3B0AARfaWQ3c3EAfgAAcHEAfgAXcHQACWNoZWNrQ29kZXh0ACNqYXZheC5mYWNlcy5jb21wb25lbnQuaHRtbC5IdG1sRm9ybXB0AARmb3JteHQAKG9yZy5hamF4NGpzZi5mcmFtZXd